In [3]:
import pandas as pd
import tabula
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
import time
import random

In [4]:
#Display solo 10 filas
pd.set_option('display.max_rows', 10)

In [5]:
#Archivo DGIL
dgil=pd.read_excel('C:/Users/claudio.pacheco/Desktop/Lista de precios de medicamentos_Diciembre_2022.xlsx',skiprows=(4), usecols=(range(5)))
dgil = dgil.iloc[:-2]
dgil.columns=dgil.columns.str.lower()
#Renmbrar penúltima y última columna
fourth_column_name = dgil.columns[3]
fifth_column_name = dgil.columns[4]
dgil.rename(columns={fourth_column_name:'pres',fifth_column_name:'precio'}, inplace=True)
#Eliminar filas vacías
dgil=dgil.dropna()
#Eliminar espacios en blanco
dgil['producto']=dgil['producto'].str.strip()
dgil

,empresa,producto,principio activo,pres,precio
0,"Abbvie Famacéuticos, S.A. de C.V.",VENCLEXTA,Venetoclax,Caja con un frasco con 120 tabletas de 100 mg.,149594.00
1,"Abbvie Famacéuticos, S.A. de C.V.",VENCLEXTA,Venetoclax,Tratamiento de inicio. Caja con 4 carteras sem...,32287.00
2,"Abbvie Famacéuticos, S.A. de C.V.",MAVYRET,Glecaprevir / Pribrenstavir,Caja con 4 cajas individuales con 7 blisters c...,96912.00
3,"Abbvie Famacéuticos, S.A. de C.V.",RIM-VOQ,Upadacitinib,Caja de cartón con 28 tabletas de 15 mg en env...,22612.88
4,"Abbvie Famacéuticos, S.A. de C.V.",SKYRIZI,Risankizumab,Caja de cartón con 2 jeringas prellenadas 0.83...,90600.00
...,...,...,...,...,...
375,"Takeda México, S.A. de C.V.",TURAZIVE,Febuxostat,Frasco con 30 tabletas 80 mg,917.00
377,"UCB de México, S.A de C.V",BRIVIACT,Brivaracetam,Caja con 28 tabletas de 25 mg,2060.00
378,"UCB de México, S.A de C.V",BRIVIACT,Brivaracetam,Caja con 28 tabletas de 50 mg,2060.00
379,"UCB de México, S.A de C.V",BRIVIACT,Brivaracetam,Caja con 28 tabletas de 100 mg,2060.00


In [6]:
dgil.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 353 entries, 0 to 380
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   empresa           353 non-null    object 
 1   producto          353 non-null    object 
 2   principio activo  353 non-null    object 
 3   pres              353 non-null    object 
 4   precio            353 non-null    float64
dtypes: float64(1), object(4)
memory usage: 16.5+ KB


In [7]:
#Leer datos de cofepris
url="https://www.gob.mx/cms/uploads/attachment/file/803597/LMR_2023-01_actualizaci_n_10_febrero_2023.pdf"


In [8]:
cofepris=tabula.read_pdf(url,pages='all',multiple_tables=True,lattice=True)

In [9]:
cofepris_df=pd.DataFrame()
for tablas in cofepris:
    cofepris_df=pd.concat([cofepris_df,tablas],ignore_index=True)

#Renombrar columna 1
cofepris_df.rename(columns={cofepris_df.columns[0]:'consecutivo',cofepris_df.columns[1]:"farmaco",
                            cofepris_df.columns[2]: "forma", cofepris_df.columns[3]:"concentra",
                            cofepris_df.columns[4]:"fraccion",cofepris_df.columns[5]:
                            "producto",cofepris_df.columns[6]:"registro",
                            cofepris_df.columns[7]:"titular",
                            cofepris_df.columns[8]:"terapeutica",
                            },inplace=True)
#Eliminar vacios de columna "consecutivo"
cofepris_df=cofepris_df.dropna(subset=['consecutivo'])
#Resetear index
cofepris_df=cofepris_df.reset_index(drop=True)  
#Eliminar espacios en blanco en columna "producto"
cofepris_df['producto']=cofepris_df['producto'].str.strip()

In [10]:
cofepris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526 entries, 0 to 1525
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   consecutivo  1526 non-null   int64 
 1   farmaco      1526 non-null   object
 2   forma        1526 non-null   object
 3   concentra    1526 non-null   object
 4   fraccion     1526 non-null   object
 5   producto     1526 non-null   object
 6   registro     1526 non-null   object
 7   titular      1526 non-null   object
 8   terapeutica  1526 non-null   object
dtypes: int64(1), object(8)
memory usage: 107.4+ KB


In [11]:
cofepris_df

,consecutivo,farmaco,forma,concentra,fraccion,producto,registro,titular,terapeutica
0,1,"ABACAVIR, Sulfato de",Tableta,300 mg,IV,ZIAGENAVIR,088M99 SSA,"GLAXO WELLCOME, S.A.",Antirretroviral.
1,2,"ABACAVIR, Sulfato de",Jarabe,2 g / 100 mL,IV,ZIAGENAVIR,087M99 SSA,"GLAXO WELLCOME, S.A.",Antirretroviral.
2,3,"ABACAVIR, Sulfato de /\rLAMIVUDINA",Tableta,600 mg / 300 mg,IV,KIVEXA,479M2005 SSA,"GLAXO WELLCOME, S.A.",Antirretroviral. Para el tratamiento de la inf...
3,4,"ABIRATERONA, Acetato de",Tableta,250 mg y 500 mg,IV,ZYTIGA,241M2012 SSA,"JANSSEN-CILAG, S.A. DE C.V.",En el tratamiento de pacientes con cáncer de p...
4,5,ACARBOSA,Tableta,50 mg y 100 mg,IV,GLUCOBAY,458M93 SSA,"BAYER DE MÉXICO, S.A. DE C.V.",Para el tratamiento de la diabetes mellitus ti...
...,...,...,...,...,...,...,...,...,...
1521,1522,ZOLPIDEM,Tableta,10mg,III,STILNOX,044M98 SSA,"SANOFI-AVENTIS DE MÉXICO,\rS.A. DE C.V.",Insomnio.
1522,1523,"ZOLPIDEM, Tartrato de",Tableta\r(Liberación prolongada),6.25 mg y 12.5 mg,III,STILNOX CR,152M2008 SSA,"SANOFI-AVENTIS DE MÉXICO,\rS.A. DE C.V.",I n d u c t o r d e l s u eño (pa...
1523,1524,"ZUCLOPENTIXOL, Acetato de",Solución\r(Inyectable),50 mg/1 mL,IV,CLOPIXOL ACUPHASE,414M96 SSA,H. LUNDBECK A/S,Antipsicótico.
1524,1525,"ZUCLOPENTIXOL, Decanoato de",Solución\r(Inyectable),200 mg/1 mL,IV,CLOPIXOL DEPOT,415M96 SSA,H. LUNDBECK A/S,Antipsicótico. Tratamiento inicial de la esqui...


In [12]:
#Cruce de ambos dataframes
cruce=pd.merge(dgil,cofepris_df,how='left',on='producto')
cruce.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 359 entries, 0 to 358
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   empresa           359 non-null    object 
 1   producto          359 non-null    object 
 2   principio activo  359 non-null    object 
 3   pres              359 non-null    object 
 4   precio            359 non-null    float64
 5   consecutivo       124 non-null    float64
 6   farmaco           124 non-null    object 
 7   forma             124 non-null    object 
 8   concentra         124 non-null    object 
 9   fraccion          124 non-null    object 
 10  registro          124 non-null    object 
 11  titular           124 non-null    object 
 12  terapeutica       124 non-null    object 
dtypes: float64(2), object(11)
memory usage: 39.3+ KB


In [13]:
#Contar valores nulos de registro
cruce['registro'].isnull().sum()

235

In [14]:
pd.set_option('display.max_rows', None)
cruce

,empresa,producto,principio activo,pres,precio,consecutivo,farmaco,forma,concentra,fraccion,registro,titular,terapeutica
0,"Abbvie Famacéuticos, S.A. de C.V.",VENCLEXTA,Venetoclax,Caja con un frasco con 120 tabletas de 100 mg.,149594.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Abbvie Famacéuticos, S.A. de C.V.",VENCLEXTA,Venetoclax,Tratamiento de inicio. Caja con 4 carteras sem...,32287.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Abbvie Famacéuticos, S.A. de C.V.",MAVYRET,Glecaprevir / Pribrenstavir,Caja con 4 cajas individuales con 7 blisters c...,96912.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Abbvie Famacéuticos, S.A. de C.V.",RIM-VOQ,Upadacitinib,Caja de cartón con 28 tabletas de 15 mg en env...,22612.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Abbvie Famacéuticos, S.A. de C.V.",SKYRIZI,Risankizumab,Caja de cartón con 2 jeringas prellenadas 0.83...,90600.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Abbvie Famacéuticos, S.A. de C.V.",SKYRIZI,Risankizumab,Pluma precargada de 150mg/1.0ml,96942.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Actelion Pharmaceuticals, S.A. de C.V.",ZAVESCA,Miglustat,Caja con 84 cápsulas de 100 mg,173526.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Amgen México, S.A. de C.V.",PROLIA,Denosumab,Caja con jeringa prellenada de 60 mg/ml,9217.68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Amgen México, S.A. de C.V.",XGEVA,Denosumab,Frasco ámpula con 120 mg / 1.7ml,10315.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"Amgen México, S.A. de C.V.",REPATHA,Evolocumab,Caja con jeringa prellenada de 140 mg/mL.,5917.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Segundo método: buscar en el catálogo de COFEPRIS

In [15]:
def cofepris(medicamento):
    #función para cambiar la denominación del medicamento
    def denominacion(option_text):
        combo = driver.find_element(By.CLASS_NAME, "form-control")
        combo_select = Select(combo)
        combo_select.select_by_visible_text(option_text)
        cuadro_busqueda = driver.find_element(By.ID, "MainContent_txtSearchValue")
        cuadro_busqueda.send_keys(medicamento)
        button = driver.find_element(By.XPATH, "//button[contains(@onclick, '__doPostBack') and @class='btn btn-primary']")
        button.click()
    #Función para extraer los elementos de la tabla
    def elementos_tabla(driver):
        table_element = driver.find_element(By.CLASS_NAME, "gridView_basic")
        table_contents = table_element.get_attribute('outerHTML')
        return table_contents
    
    #Código para extraer los datos de la página
    url = "https://tramiteselectronicos02.cofepris.gob.mx/BuscadorPublicoRegistrosSanitarios/BusquedaRegistroSanitario.aspx"
    options = webdriver.ChromeOptions()
    options.add_argument('--incognito')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-cache')
    options.add_argument('--disable-cookies')
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service("C:/chromedriver.exe"), options=options)
    driver.get(url)
    #Denominación Distintiva
    denominacion("Denominación Distintiva")

    try:
        table_contents = elementos_tabla(driver)
    except NoSuchElementException:
        #Denominación Genérica
        denominacion("Denominación Genérica")
        try:
            table_contents = elementos_tabla(driver)
        except NoSuchElementException:
            driver.quit()
            return pd.DataFrame({'marca': [], 'generica': [], 'laboratorio': [], 'fabricante': []})

    soup = BeautifulSoup(table_contents, 'html.parser')
    registro=[]
    marca = []
    generica = []
    indica = []
    lab = []
    fabricante = []
    principio=[]

    for span in soup.find_all('span'):
        if span.has_attr('id'):
            if span['id'].startswith('MainContent_GV_RegistroSanitario_lblNumeroRegistro'):
                registro.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblDenominacionDistintiva'):
                marca.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblDenominacionGenerica'):
                generica.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblTitularRegistro'):
                lab.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblFabricanteMedicamento'):
                fabricante.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblPrincipioActivo'):
                principio.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblIndicacionTerapeutica'):
                indica.append(span.text)

    df = pd.DataFrame({'reg_san':registro,'marca': marca, 'den_gen': generica, 'titular': lab, 'principio': principio,'indicacion':indica})
    df = df.apply(lambda x: x.astype(str).str.lower())
    df = df.apply(lambda x: x.astype(str).str.replace('\n', ''))
    driver.quit()
    return df


In [16]:
#hacer una muestra aleatoria de 10 medicamentos
muestra=dgil.sample(10)
muestra

,empresa,producto,principio activo,pres,precio
191,"Janssen-Cilag de México, S. de R.L. de C.V.",ZEPENDO-OPSUMIT,Macitentan,28 tabletas de 10mg,69911.60
327,"Sanofi-Aventis de México, S.A. de C.V.",LYXUMIA,Lixisenatida,Caja c/1 cartucho con 3ml en dispositivo desec...,999.00
154,"Grimann, S.A DE C.V.",Xiliarx,Vildagliptina,Caja con 56 comprimidos con 50 mg,991.00
173,"Janssen-Cilag de México, S. de R.L. de C.V.",IMBRUVICA,Ibrutinib,Caja con 120 tabletas de 140 mg,157929.72
243,"Novartis Farmacéutica, S.A. de C.V.",JAKAVI,Ruxolitinib,Frasco con 60 tabletas de 5mg,46250.00
187,"Janssen-Cilag de México, S. de R.L. de C.V.",STELARA,Ustekinumab,Caja con vial 90mg,73349.96
230,"Novartis Farmacéutica, S.A. de C.V.",GALVUS,Vildagliptina,Caja con 28 tabletas con 50 mg.,511.00
372,"Takeda México, S.A. de C.V.",INCRESINA DUO,Alogliptina/Metformina,Caja con 56 tabletas de 12.5/850 mg,1071.00
332,"Merck Sharp & Dohme Comercializadora, S. de R....",ISENTRESS,Raltegravir,Caja con 60 tabletas de 600 mg,31304.05
192,"Janssen-Cilag de México, S. de R.L. de C.V.",UPTRAVI,Selexipag,60 tabletas de 200 mcg,63556.00


In [17]:
#Buscar medicamentos en cofepris
busqueda=pd.DataFrame()
for i in muestra['producto']:
    busqueda=pd.concat([busqueda,cofepris(i)],ignore_index=True)

In [18]:
busqueda

,marca,generica,laboratorio,fabricante,reg_san,den_gen,titular,principio,indicacion
0,imbruvica,NaN,NaN,NaN,003m2015 ssa,ibrutinib,"janssen-cilag, s.a. de c.v.",l01el01 ibrutinib,linfoma de células de manto (lcm): pacientes a...
1,imbruvica,NaN,NaN,NaN,217m2022 ssa,ibrutinib,janssen -cilag s.a de c.v,l01el01 ibrutinib,linfoma de células de manto (lcm): pacientes a...
2,jakavi,NaN,NaN,NaN,123300el870007,ruxolitinib,,,tratamiento de la mielofibrosis idiopática cró...
3,stelara iv,NaN,NaN,NaN,358m2018 ssa,ustekinumab,"janssen-cilag, s.a. de c.v.",l04ac5 ustekinumab,enfermedad de crohn.tratamiento de la enfermed...
4,stelara,NaN,NaN,NaN,246m2009 ssa,ustekinumab,janssen -cilag s.a de c.v,l04ac5 ustekinumab,"psoriasis en placatratamiento de la psoriasis,..."
5,galvus,NaN,NaN,NaN,045m2007 ssa,vildagliptina,,a10bh02 vildagliptina.,para el tratamiento de la diabetes mellitus ti...
6,galvus met,NaN,NaN,NaN,180m2008 ssa,vildagliptina / metformina,,a10ba02 metformina.,para el tratamiento de la diabetes mellitus ti...
7,incresina duo,NaN,NaN,NaN,269m2014 ssa,alogliptina / metformina,,a10bd99 otros principios activos. combinacion...,está indicado como tratamiento complementario ...
8,isentress,NaN,NaN,NaN,308m2007 ssa,raltegravir,merck sharp & dohme b.v.,j05ax01 otros antivirales.,indicado en combinación de otros antirretrovir...
9,uptravi,NaN,NaN,NaN,205m2018 ssa,selexipag,janssen -cilag s.a de c.v,b01ac27 selexipag,tratamiento a largo plazo de la hipertensión a...
